In [7]:
# Load data
import pandas as pd
import numpy as np
url = "SMSSpamCollection.txt"
df = pd.read_csv(url, sep='\t', header=None )
df.rename(columns={0:'type',1:'mail'}, inplace=True)
df


,type,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [9]:
df.describe()

,type,mail
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [16]:
df_ham=df[df['type']=='ham']

df_ham['split']=df_ham['mail'].str.split()
df_ham

C:\Users\MOHAM\AppData\Local\Temp\ipykernel_14956\1546412889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ham['split']=df_ham['mail'].str.split()


,type,mail,split
0,ham,"Go until jurong point, crazy.. Available only ...","[Go, until, jurong, point,, crazy.., Available..."
1,ham,Ok lar... Joking wif u oni...,"[Ok, lar..., Joking, wif, u, oni...]"
3,ham,U dun say so early hor... U c already then say...,"[U, dun, say, so, early, hor..., U, c, already..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[Nah, I, don't, think, he, goes, to, usf,, he,..."
6,ham,Even my brother is not like to speak with me. ...,"[Even, my, brother, is, not, like, to, speak, ..."
...,...,...,...
5565,ham,Huh y lei...,"[Huh, y, lei...]"
5568,ham,Will ü b going to esplanade fr home?,"[Will, ü, b, going, to, esplanade, fr, home?]"
5569,ham,"Pity, * was in mood for that. So...any other s...","[Pity,, *, was, in, mood, for, that., So...any..."
5570,ham,The guy did some bitching but I acted like i'd...,"[The, guy, did, some, bitching, but, I, acted,..."


In [13]:
df_spam=df[df['type']=='spam']
df_spam

,type,mail
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load data
url = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip"
df_music = pd.read_csv(url)

# Train test split
X = df_music.drop('genre', axis=1)
y = df_music['genre']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Categorical variables
column_cat = X.select_dtypes(include='object').columns.drop(['track_name', 'track_id'])#['key']
transfo_cat = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Numerical variables
column_num = X.select_dtypes(include=np.number).columns#['acousticness']
transfo_num = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='median')),
    ('scaling', StandardScaler())
])

# Class ColumnTransformer : apply alls steps on the whole dataset
preparation = ColumnTransformer(
    transformers=[
        ('data_cat', transfo_cat , column_cat),
        ('data_num', transfo_num , column_num)
    ])

# Creation of model : a ready to use pipeline for ML process
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(3)
model_lm = Pipeline(steps=[('preparation', preparation),
                        ('model', classifier)])

# Fit the model
model_lm.fit(X_train, y_train)

# Predict on test data
y_pred = model_lm.predict(X_test)

# Performance of the model
print("Accuracy:", accuracy_score(y_test, y_pred))